In [1]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [2]:
import json
import random
from glob import glob

In [4]:
data = []

for f in glob('/home/husein/ssd3/instructions/function-calls*.jsonl'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            texts = []
            for no, d in enumerate(l['conversations']):

                if d['role'] == 'user':
                    t = '<manusia>: '
                    if random.random() > 0.5:
                        c = 'content'
                    else:
                        c = 'content'
                else:
                    if 'functioncall' in d['content']:
                        c = 'content'
                    else:
                        if 'functioncall' in l['conversations'][no - 1]['content']:
                            continue
                        c = 'content'
                    t = '<bot>: '

                if d[c] is None:
                    break

                if 'functioncall' in d[c]:
                    d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

                n = d[c]
                n = n.strip()
                if len(n) < 3:
                    break
                t = t + n
                texts.append(t)
                
            if not len(texts):
                continue
            while len(texts) and texts[-1].startswith('<manusia>'):
                texts = texts[:-1]
            if not len(texts):
                continue
            if not texts[0].startswith('<manusia>'):
                continue

            data.append({
                'prompt_input': None,
                'input': '\n'.join(texts),
                'output': None,
                'function_call': l['function_call'],
            })
            
len(data)

196774

In [5]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [6]:
print(generate_and_tokenize_prompt(data[-4])['text'])

<s>
[FUNCTIONCALL]
{
 "name": "hitung_imt",
 "description": "Hitung Indeks Massa Tubuh (IMT)",
 "parameters": {
 "type": "object",
 "properties": {
 "berat": {
 "type": "number",
 "description": "Berat dalam kilogram"
 },
 "tinggi": {
 "type": "number",
 "description": "Tinggi dalam meter"
 }
 },
 "required": [
 "berat",
 "tinggi"
 ]
 }
}
[INST] Hai, saya ingin mengira IMT saya. Saya berat 70 kilogram dan tinggi saya 1.75 meter. [/INST] <functioncall> {'name': 'hitung_imt', 'arguments': {'berat': 70, 'tinggi': 1.75}}</s> 


In [8]:
from tqdm import tqdm

for d in data:
    generate_and_tokenize_prompt(d)['text']

In [9]:
print(generate_and_tokenize_prompt(data[1])['text'])

<s>
[FUNCTIONCALL]
{"name": "GeneratePurchaseOrder","description":"Fungsi ini digunakan untuk menghasilkan pesanan pembelian. Ia menerima maklumat produk, kuantiti, harga dan alamat penghantaran. Setiap produk perlu disertakan dengan kuantiti yang dikehendaki.","parameters":{"type":"object","properties":{"products":{"type":"array","items":{"type":"object","properties":{"product_name":{"type":"string","description":"Nama produk yang dipesan."},"quantity":{"type":"integer","description":"Kuantiti produk yang dikehendaki."},"price":{"type":"number","description":"Harga bagi satu unit produk."}},"required":["product_name","quantity","price"]}},"delivery_address":{"type":"string","description":"Alamat lengkap untuk penghantaran."}},"required":["products","delivery_address"]}}
[INST] Saya ingin membuat pesanan pembelian. Produk yang saya perlukan ialah:
1. T-shirt, kuantiti 5, harga 25.50
2. Kasut sukan, kuantiti 2, harga 80.00
Alamat penghantaran: No. 123, Jalan Merdeka, 68000 Ampang. [/INS

In [10]:
import json

with open('prepared-chatgpt-malay-function-call.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')

In [11]:
!wc -l prepared-chatgpt-malay-function-call.jsonl

196774 prepared-chatgpt-malay-function-call.jsonl
